# Enfermedades mentales en tecnología
## Facultad TIC Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Nathalia Andrea Ortega Gómez

**ID:** 000200550

**Email:** nathalia.ortega@upb.edu.co

**Fecha de Entrega: 22 de Octubre de 2020**

## Introducción

Cada año, [OSMI](https://osmihelp.org/) realiza una encuesta a gran escala sobre la prevalencia y las actitudes hacia la salud mentall de las personas que trabajan en la industria tecnológica.

Las posibles salidas para la predicción son:
- Yes (tiene actualmente una enfermedad mental)
- No (No tiene una enfermedad mental)
- Don't Know (No sabe si tiene una enfermedad mental)

## Objetivo principal
** Predecir qué empleados tienen más probabilidad de padecer una enfermedad mental. **

## Información de los datos


# Importar librerías

In [1]:
pip install pandas


You should consider upgrading via the 'C:\Users\natya\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\natya\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\natya\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np

In [5]:
import matplotlib.pyplot as pyplot

# Cargar los datos

In [6]:
data_raw = pd.read_csv("../data/raw/enfermedadesMentales2017-2018-clean-columns.csv")

# Descripción y limpieza de los datos

In [7]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 98 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   #                                                           1173 non-null   object 
 1   independiente/emprendedor                                   1173 non-null   int64  
 2   No_empleados                                                1004 non-null   object 
 3   compania_tecnologia                                         1004 non-null   float64
 4   cargo_tecnologia                                            1004 non-null   float64
 5   cubrimiento_EM                                              1004 non-null   object 
 6   concimiento_cubrimiento_EM                                  900 non-null    object 
 7   charla_EM                                                   1004 non-null   object 
 8 

# Conclusiones
La insdutria tecnologica se caracteriza por su innovación, resiliencia y una cultura que fomente la alta productividad; es un ritmo de juego rápido, está bajo una intensa presión para tener exito y mantenerse a la vanguardia. Debido a esto, los empleados de la insutria tecnologica operan bajo un nivel de estrés alto con horarios anormales, plazos ajustados, múltiples responsabilidades y disponibilidad en cualquier momento del día ([IQmetrix](https://https://www.iqmetrix.com/blog/lets-talk-its-time-to-get-serious-about-mental-illness-in-tech)).

De acuerdo con los resultados de [OSMI 2016](https://osmihelp.org/blog/the-osmi-2016-survey-results-are-part-of-data), el 51% de los profesionales que trabajan en tecnología fueron diagnosticados con una enfermedad mental, su verdadero problema radica en lo invisible de una enfermedad mental, se convierte en una batalla privada y es dificil saber cuando las personas necesitan ayuda.

A pesar de todo lo anterior, los estigmas a las enfermedades mentales han ido cambiado con el pasar de los años, existen más espacios donde hablar estos temas y mejor cubrimiento para las enfermedades y trastornos mentales en los planes de salud ([Forbes](https://www.forbes.com/sites/markhall/2020/02/04/mental-health-mainstream/#cec59ac5cd5e)), entendendiendo así la importancia de la salud mental tanto en el desempeño del trabajo, como el mejoramiento de la calidad de vida . [En 2018, un estudio](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5891372/) reveló que el 86% de los empleados que reciben apoyo y atención a las enfermedades mentales por parte de sus empresas, reportan un mejor desempeño y menor ausencia en sus trabajos.